gs://msca-bdp-tweets/final_project. 

## Load lib

In [1]:
# Update environment (if needed)
# !sudo apt update

# Download and install Java
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install Apache Spark 3.0.3 with Hadoop 3.2
!wget -nc -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

# Unzip the folder
!tar xf  spark-3.0.3-bin-hadoop3.2.tgz

# Install findspark library that will locate Spark on the system
!pip install -q findspark

E: Package 'openjdk-8-jdk-headless' has no installation candidate


In [2]:
# Setting the environment variables, to enable running PySpark in Colab environment.

import os
import shutil
from itertools import islice
import requests

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop3.2"

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.reset_option('display.max_rows')

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#from pyspark.sql import functions as F
from pyspark.sql.types import *

from pyspark.sql.functions import mean, stddev

# Create spark_session
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

import sys
print(sys.version)
print(spark.version)

3.8.5 | packaged by conda-forge | (default, Aug 29 2020, 01:22:49) 
[GCC 7.5.0]
3.0.1


In [4]:
import re
from pyspark.ml.feature import MinHashLSH
from pyspark.ml.feature import CountVectorizer,  IDF, CountVectorizerModel, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Read Data

In [5]:
import os
import subprocess
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [6]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [7]:
directory = 'gs://msca-bdp-tweets/final_project/'
file = '*.json'
path = directory + file
#path = directory

In [8]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

156.2 G  156.2 G  gs://msca-bdp-tweets/final_project



In [9]:
%%time

tweets_df = spark.read.json(path)

CPU times: user 36.5 ms, sys: 9.21 ms, total: 45.8 ms
Wall time: 4min 26s


## EDA

In [10]:
%%time

tweets_df.count()

CPU times: user 10.6 ms, sys: 474 µs, total: 11.1 ms
Wall time: 1min 3s


25191000

In [27]:
pd.set_option("max_colwidth", 1000)
tweets_df.describe()

summary,contributors,created_at,favorite_count,filter_level,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,lang,quote_count,quoted_status_id,quoted_status_id_str,reply_count,retweet_count,source,text,timestamp_ms
count,0,25191000,25191000,25191000,25191000,25191000,3332829,3181701,3181701,3332829,3332829,25191000,25191000,5238622,5238622,25191000,25191000,25191000,25191000,25191000
mean,null,null,0.0,null,1.453838088871457...,1.453838088871457...,Infinity,1.453092037554489...,1.453092037554489...,4.482970009158758...,4.482970009158758...,null,0.0,1.451420487744112...,1.451420487744112...,0.0,0.0,null,null,1.635456986049065...
stddev,null,null,0.0,null,2.980993718097450...,2.980993718097450...,NaN,1.403312255886593E16,1.403312255886593E16,5.858283007481441...,5.858283007481441...,null,0.0,1.934290822993775...,1.934290822993775...,0.0,0.0,null,null,7.107242865795615E8
min,null,Fri Nov 05 00:00:...,0,low,1448850337247367173,1448850337247367173,000000001404072,15754281,1001725136045793280,12,1000002715999784961,en,0,1403739068,1000324596757942272,0,0,"<a href=""HTTP://b...",! An extremely s...,1634267813394
max,null,Wed Oct 27 23:59:...,0,low,1459021442469666820,1459021442469666820,zzzzzz_cz17,1459021295652204545,999297742034923520,1458982120475398146,999996275184619520,en,0,1459020844458520587,999652465548054533,0,0,"<a href=""https://...",🫖After a long CO...,1636692793849


In [11]:
tweets_df.select('is_quote_status','retweeted_status','text','timestamp_ms','truncated','user','withheld_in_countries').show(10)
tweets_df.select('user').show(10)


+---------------+--------------------+--------------------+-------------+---------+--------------------+---------------------+
|is_quote_status|    retweeted_status|                text| timestamp_ms|truncated|                user|withheld_in_countries|
+---------------+--------------------+--------------------+-------------+---------+--------------------+---------------------+
|           true|[,, Sat Oct 30 02...|RT @LilNasX: y’al...|1635562341417|    false|[false, Sun May 2...|                 null|
|          false|                null|“At a political f...|1635562341604|     true|[false, Mon Feb 2...|                 null|
|          false|                null|@GarrAarghHrumph ...|1635562341946|    false|[false, Mon May 2...|                 null|
|           true|[,, Sat Oct 30 02...|RT @Denise_Old_La...|1635562342062|    false|[false, Sun Mar 2...|                 null|
|           true|[,, Sat Oct 30 02...|RT @LilNasX: y’al...|1635562342041|    false|[false, Fri May 0...|       

## Filter df based on covid

In [11]:
tweets_df_cleaned=tweets_df.select('coordinates','created_at', 'geo','id_str','place', 'lang',
                                   'is_quote_status','quote_count','quoted_status','quoted_status_id_str',
                                   'retweeted','retweet_count','retweeted_status','text',
                                   'timestamp_ms','user')

In [12]:
tweets_df_covid=tweets_df_cleaned.filter(tweets_df_cleaned.text.contains('covid')|tweets_df_cleaned.text.contains('COVID')|tweets_df_cleaned.text.contains('Covid'))


In [13]:
tweets_df_covid.count()

14948838

In [16]:
df_concat=tweets_df_covid.dropDuplicates()

In [17]:
df_concat_5000=df_concat.limit(5000)

In [22]:
#df_concat_5000.toPandas().to_csv('df_concat_5000.csv')

## Save 5000 samples to parquet

In [35]:
output_directory = 'gs://msca-bdp-students-bucket/shared_data/charleneyu/df5000'
df_concat_5000.write.format("parquet").\
mode('overwrite').\
save(output_directory)

In [22]:
df_concat.filter(df_concat.text.isNull()).count()

0

In [41]:
df_concat_5000.filter(df_concat_5000.retweeted_status.isNotNull()).count()

3561

In [ ]:
df_concat_5000.filter(df_concat_5000.quoted_status.isNotNull()).count()

In [ ]:
df_concat_5000.show(2)

+-----------+---------+------------+----+-------------------+---------------+----+------------------+-----------+-------------+--------------------+---------+--------------------+--------------------+-------------+---------+--------------------+
|coordinates|favorited|filter_level| geo|             id_str|is_quote_status|lang|possibly_sensitive|quote_count|quoted_status|quoted_status_id_str|retweeted|    retweeted_status|                text| timestamp_ms|truncated|                user|
+-----------+---------+------------+----+-------------------+---------------+----+------------------+-----------+-------------+--------------------+---------+--------------------+--------------------+-------------+---------+--------------------+
|       null|    false|         low|null|1450702446930677768|          false|  en|              null|          0|         null|                null|    false|                null|Hi @SimonMcCoyTV ...|1634709390753|     true|[false, Sun Nov 2...|
|       null|   

## Flatten columns

In [27]:
df_user=df_concat_5000.select('id_str','retweeted','retweet_count','user','retweeted_status').\
withColumn("retweeted_or_not", when(df_concat_5000.retweeted_status.isNull(), 'original').otherwise('retweet'))


In [ ]:
df_user=df_user.select('id_str','retweeted','retweet_count','user','retweeted_or_not')

df_user_f=flatten_df(df_user)
df_user_f.toPandas().to_csv('df_user_5000.csv')

In [15]:
df_quote=df_concat_5000[['id_str','is_quote_status','quote_count','quoted_status','quoted_status_id_str']]
df_retweeted=df_concat_5000[[ 'id_str','retweeted','retweet_count','retweeted_status']]
df_user=df_concat_5000[['id_str','retweeted','retweet_count','user']]
df_geocoord=df_concat_5000.select('id_str','geo', 'place','coordinates').filter(df_concat_5000.place.isNotNull())
df_time=df_concat_5000[['id_str','created_at']]
df_text=df_concat_5000[['id_str','text']]

In [34]:
df_text_5000=df_text
output_directory = 'gs://msca-bdp-students-bucket/shared_data/charleneyu/df_text'
df_text_5000.write.format("json").\
mode('overwrite').\
save(output_directory) 


In [14]:
import pyspark.sql.functions as F

def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [F.col(nc+'.'+c).alias(nc+'_'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [26]:
df_user_f=flatten_df(df_user)
df_geocoord_f=flatten_df(df_geocoord)

In [ ]:
df_time.toPandas().to_csv('df_time_5000.csv')
df_user_f.toPandas().to_csv('df_user_5000.csv')

In [28]:
df_geocoord_f.toPandas().to_csv('df_geocoord_5000.csv')

In [ ]:
df_quote.toPandas().to_csv('df_quote_5000_og.csv')
df_retweeted.toPandas().to_csv('df_retweeted_5000_og.csv')

In [ ]:
df_retweeted_tweets=df_retweeted.filter(df_retweeted.retweeted_status.isNotNull())
df_retweeted_tweets_f=flatten_df(df_retweeted_tweets)
df_retweeted_tweets_f.toPandas().to_csv('df_retweeted_tweets_5000.csv')

In [ ]:
#df_quote_samp=df_quote.filter(df_quote.is_quote_status == 'True')
df_quote_f=flatten_df(df_quote)
df_quote_f.toPandas().to_csv('df_quote_5000.csv')


In [29]:
df_user.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- user_contributors_enabled: boolean (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- user_default_profile: boolean (nullable = true)
 |-- user_default_profile_image: boolean (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_favourites_count: long (nullable = true)
 |-- user_follow_request_sent: string (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_following: string (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_geo_enabled: boolean (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_id_str: string (nullable = true)
 |-- user_is_translator: boolean (nullable = true)
 |-- user_lang: string (nullable = true)
 |-- user_listed_count: long (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_notifications: string (nullable = true)
 |-- user_profile_background_color: s

In [27]:
df_geocoord.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- geo_coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- geo_type: string (nullable = true)
 |-- place_bounding_box: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: array (containsNull = true)
 |    |    |    |-- element: array (containsNull = true)
 |    |    |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- place_country: string (nullable = true)
 |-- place_country_code: string (nullable = true)
 |-- place_full_name: string (nullable = true)
 |-- place_id: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_place_type: string (nullable = true)
 |-- place_url: string (nullable = true)
 |-- coordinates_coordinates: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- coordinates_type: string (nullable = true)



In [ ]:
df_user.groupBy('user_id_str').agg(count('id_str')).orderBy(count('id_str'), ascending=False).limit(5).show(5)

+-------------------+-------------+
|        user_id_str|count(id_str)|
+-------------------+-------------+
| 952247125185720320|            6|
|1257596262649561089|            3|
|1269612893529444354|            3|
|          480875170|            3|
|         3026454417|            2|
+-------------------+-------------+



In [14]:
tweets_df.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: bo